**01. 데이터 로딩**

In [1]:
# 라이브러리
import numpy as np
import pandas as pd

# 데이터 로딩
dataset = pd.read_csv('./final_dataset.csv')

# 불필요한 열 제거
del dataset['Unnamed: 0']

# 데이터 보기
dataset

,open,high,low,close,volume,score,index,probability,l1,l2,l3,l4,lgap,lrate,open_high,open_low,open_close,zone
0,-4.334975,-2.758621,-2.760736,-1.017294,12.589118,1.329195,0.887480,47.114,-1.916462,-1.901902,-1.886792,-1.871102,6.293419,6.0,0.016478,-0.020597,0.002060,+3 ~ +10
1,1.338826,0.405268,-8.517350,-6.680370,13.340528,1.641324,0.672944,27.960,1.202405,-3.112245,-7.588358,-12.235169,7.504392,20.0,0.007114,-0.115854,-0.077236,-3 ~ -1
2,-11.077236,-4.339051,0.574713,3.524229,12.721548,1.597365,0.631272,23.643,-2.920792,-0.579252,2.080990,5.129753,6.999422,12.0,0.083429,0.000000,0.074286,-3 ~ -1
3,5.257143,3.059072,5.257143,3.829787,12.550566,1.597365,0.631272,23.643,2.447731,3.442797,4.517906,5.683123,6.734592,9.0,0.060803,0.000000,0.059718,-1 ~ +1
4,6.948969,3.377687,5.320304,-0.102459,12.772218,1.597365,0.631272,23.643,0.796416,1.638505,2.530311,3.476372,6.398595,6.0,0.025381,-0.015228,-0.010152,-3 ~ -1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901794,-0.371496,-0.468227,-0.169205,0.337838,6.994850,7.957877,2.397895,79.010,-0.217210,-0.067227,0.084531,0.238095,5.278115,1.0,0.008814,0.000000,0.006780,-1 ~ +1
1901795,0.677966,0.840054,0.271186,0.000000,8.268988,7.913155,3.044522,79.751,0.703282,0.420451,0.135135,-0.152698,5.779199,2.0,0.010438,-0.004040,0.000000,-1 ~ +1
1901796,-1.043771,-1.066311,-0.642326,-0.033670,7.442493,7.909489,3.044522,79.751,-0.764882,-0.552671,-0.337382,-0.118946,5.420535,2.0,0.010208,0.000000,0.010208,+1 ~ +3
1901797,-0.306227,-1.111485,-0.918680,-1.481980,6.194405,7.896553,1.945910,77.297,-1.390751,-1.296733,-1.201760,-1.105818,5.198497,1.0,0.002048,-0.006143,-0.001706,-3 ~ -1


**02. Train 데이터와 Test 데이터 분리**

In [2]:
# 구역 숫자로 변환
def zone(x) :
    if x == '-30 ~ -10' : return 0
    elif x == '-10 ~ -3' : return 1
    elif x == '-3 ~ -1' : return 2
    elif x == '-1 ~ +1' : return 3
    elif x == '+1 ~ +3' : return 4
    elif x == '+3 ~ +10' : return 5
    elif x == '+10 ~ +20' : return 6
    else : return 7
        
dataset['zone'] = dataset['zone'].apply(lambda x : zone(x))

# 어레이로 데이터 변경
dataset = np.array(dataset)

# 80%를 학습하는데 이용
train_idx = int(len(dataset) * 0.8)
train_X, train_Y = dataset[:train_idx, :-1], dataset[:train_idx, -1]
test_X, test_Y = dataset[train_idx:, :-1], dataset[train_idx:, -1]

# 카테고리 지정
import tensorflow as tf
train_Y = tf.keras.utils.to_categorical(train_Y, num_classes=8)
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes=8)

**03. Model Build**

In [3]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(units=10,activation='relu',input_shape=(train_X.shape[1],)),
                             tf.keras.layers.Dense(units=10,activation='relu'),
                             tf.keras.layers.Dense(units=8, activation='softmax')
])
        
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                180       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
Total params: 378
Trainable params: 378
Non-trainable params: 0
_________________________________________________________________


**04. Learning**

In [4]:
history = model.fit(train_X, train_Y, epochs=25, batch_size=4096, validation_split=0.25)

Train on 1141079 samples, validate on 380360 samples
Epoch 1/25
1141079/1141079 [==============================] - 2s 2us/sample - loss: 1.3526 - accuracy: 0.4580 - val_loss: 1.2240 - val_accuracy: 0.4695
Epoch 2/25
1141079/1141079 [==============================] - 1s 1us/sample - loss: 1.2272 - accuracy: 0.4708 - val_loss: 1.2209 - val_accuracy: 0.4705
Epoch 3/25
1141079/1141079 [==============================] - 1s 1us/sample - loss: 1.2228 - accuracy: 0.4717 - val_loss: 1.2212 - val_accuracy: 0.4710
Epoch 4/25
1141079/1141079 [==============================] - 1s 1us/sample - loss: 1.2223 - accuracy: 0.4717 - val_loss: 1.2204 - val_accuracy: 0.4704
Epoch 5/25
1141079/1141079 [==============================] - 1s 1us/sample - loss: 1.2230 - accuracy: 0.4718 - val_loss: 1.2237 - val_accuracy: 0.4702
Epoch 6/25
1141079/1141079 [==============================] - 1s 1us/sample - loss: 1.2219 - accuracy: 0.4723 - val_loss: 1.2223 - val_accuracy: 0.4714
Epoch 7/25
1141079/1141079 [=======

**05. Test Evaluate**

In [5]:
model.evaluate(test_X, test_Y)

380360/380360 [==============================] - 9s 23us/sample - loss: 1.0913 - accuracy: 0.5472


[1.0912521941038271, 0.54718685]